In [2]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import skimage
import skimage.filters
import skimage.transform
import skimage.draw
from tqdm.notebook import tqdm

In [3]:
def create_heatmaps(keypoints, input_shape = (256, 256), output_shape = (64, 64)):
    """ GIVEN ONE OF THE ROWS IN THE CSV, CREATES THE 17 CORRESPONDING HEATMAPSS """
    keypoints = keypoints[1:] # We dont need the ID of the image

    x_val = keypoints[::3]
    y_val = keypoints[1::3]
    v_val = keypoints[2::3]

    res_arr = []

    for x, y, v in zip(x_val, y_val, v_val):
        x, y, v = int(x), int(y), int(v)
        res = np.zeros(input_shape)

        if (x >= 0 and x <= input_shape[1] and y >= 0 and y <= input_shape[0]):
            res[y - 1, x - 1] = 1

        res = skimage.transform.resize(res, output_shape)
        
        if (v == 1):
            res = skimage.filters.gaussian(res, sigma = 1)
        elif (v == 2):
            res = skimage.filters.gaussian(res, sigma = 0.5)

        res_arr.append(res)

    return res_arr

In [4]:
TRAIN_LABELS_PATH = "D:/bsc_data/train/outputs.txt"

HEADER = ["ID"]
for i in range(17):
    HEADER.append("x{}".format(i))
    HEADER.append("y{}".format(i))
    HEADER.append("v{}".format(i))

train_labels = pd.read_csv(TRAIN_LABELS_PATH, delimiter = ",", names = HEADER)


In [5]:
for index, row in tqdm(train_labels.iterrows(), desc = "ROW", total = train_labels.shape[0]):
    heatmaps = create_heatmaps(row.values)

    os.mkdir("D:/bsc_data/train/heatmaps/" + row["ID"])

    for i, heatmap in enumerate(heatmaps):
        plt.imsave("D:/bsc_data/train/heatmaps/" + row["ID"] + "/" + str(i) + ".png", heatmap)